In [7]:
import pandas as pd
import re
from bertopic import BERTopic

In [8]:
tweet = pd.read_csv('cleaned_kp.csv')
tweet = tweet.dropna(subset=['full_text'])

In [9]:
tweet.full_text = tweet.apply(lambda row: re.sub(r"http\S+", "", row.full_text).lower(), 1)
tweet.full_text = tweet.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.full_text.split())), 1)
tweet.full_text = tweet.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.full_text).split()), 1)

In [10]:
tweet = tweet.loc[(tweet.full_text != ""), :]
timestamps = tweet.created_at.to_list()

In [11]:
tweets = tweet.full_text.to_list()

In [12]:
topic_model = BERTopic(verbose=True)
topics, probs = topic_model.fit_transform(tweets)

2024-05-15 14:03:07,426 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/264 [00:00<?, ?it/s]

2024-05-15 14:03:48,617 - BERTopic - Embedding - Completed ✓
2024-05-15 14:03:48,618 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-15 14:04:18,561 - BERTopic - Dimensionality - Completed ✓
2024-05-15 14:04:18,562 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-15 14:04:18,866 - BERTopic - Cluster - Completed ✓
2024-05-15 14:04:18,872 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-15 14:04:19,287 - BERTopic - Representation - Completed ✓


In [13]:
topics_over_time = topic_model.topics_over_time(tweets, timestamps, nr_bins=20)

20it [00:02,  9.92it/s]


In [24]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [19]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2780,-1_jadi_indonesia_menteri_ajar,"[jadi, indonesia, menteri, ajar, baru, nadiem,...",[ini ubah lama masa pandemi covid perintah lal...
1,0,213,0_guru_pg_siswa_murid,"[guru, pg, siswa, murid, himpun, buku, nilai, ...",[rencana perintah lalu menteri budaya kemendik...
2,1,178,1_enggak_kalo_sih_gue,"[enggak, kalo, sih, gue, bikin, un, mau, lbp, ...",[bu dengar menteri baru hapus un pak sih benar...
3,2,167,2_jelasarahnya_semakindekat_semakinpakbowo_vis...,"[jelasarahnya, semakindekat, semakinpakbowo, v...",[mantab nih sob harap bapak prabowo naik stand...
4,3,126,3_dinas_kota_kepala_pasuruan,"[dinas, kota, kepala, pasuruan, kabupaten, kan...",[polres pasuruan kota guru peduli cerdas anak ...
...,...,...,...,...,...
158,157,11,157_student_loan_skema_amerika,"[student, loan, skema, amerika, debtloan, neol...",[ngeri uang kuliah naik x lipat inflasi kalo s...
159,158,11,158_politis_ajang_politisasi_peraturanpendidikan,"[politis, ajang, politisasi, peraturanpendidik...",[inti kalo mau bagus politis bagus enggak busu...
160,159,10,159_bijaksana_kwalitan_segera_awal,"[bijaksana, kwalitan, segera, awal, bid, moga,...","[moga perintah segera ambil bijaksana, baru aw..."
161,160,10,160_penting_undur_nyawa_netijen,"[penting, undur, nyawa, netijen, far, dipungki...",[point penting tinggi salah fokus sama komen n...


In [23]:
topic_model.get_topics()

{-1: [('jadi', 0.005563176419369394),
  ('indonesia', 0.005523398220875378),
  ('menteri', 0.004961909093024548),
  ('ajar', 0.004786270918264747),
  ('baru', 0.004766504127676478),
  ('nadiem', 0.004598247191618029),
  ('satu', 0.004508614746540037),
  ('perintah', 0.0044418574644911),
  ('anak', 0.0042890410419097824),
  ('sama', 0.004261081446388304)],
 0: [('guru', 0.05421286810236471),
  ('pg', 0.011616805820918428),
  ('siswa', 0.011508452731238912),
  ('murid', 0.011499636933667713),
  ('himpun', 0.01066259044420365),
  ('buku', 0.010563084748720595),
  ('nilai', 0.008814558309153633),
  ('mang', 0.008621012221829679),
  ('sejahtera', 0.008571201966459309),
  ('moga', 0.008203121456264532)],
 1: [('enggak', 0.05158888797416938),
  ('kalo', 0.019313115821261974),
  ('sih', 0.014138101239562533),
  ('gue', 0.01343912290214853),
  ('bikin', 0.01334279448064898),
  ('un', 0.012372934805823917),
  ('mau', 0.0122394507682596),
  ('lbp', 0.011847126001441426),
  ('kayak', 0.01121923450